In [1]:
import face_recognition
import cv2
import numpy as np
import pandas as pd
import requests

def mark_attendance(ids):
    """
     It takes id , save the ids in attendance.csv file and send them notification on their 
     phone number .
     
     -Args: ids
     -return: None
    """
    
    df = pd.DataFrame({
        'Roll Number' : ids
    })
    df.to_csv('attendance.csv')
    
    #saving the roll number and dropping un necessary columns
    unique_phone_ = []
    new_df = pd.read_csv('attendance.csv')
    columns_list = np.array(new_df.columns)
    drop_col = []
    for col in columns_list:
        if "Unnamed:" in  col:
            drop_col.append(col)
    new_df.drop(drop_col,axis = 1,inplace=True)
    new_df.fillna(0,inplace=True)
    new_df.to_csv('attendance.csv',index=False)
    
    #sending them notification using fast 2 sms service
    df = pd.read_csv('students.csv')
    phone_numbers = []
    names = []
    for idi in ids:
        if int(idi) in df['Roll Number'].unique():
            phone_numbers.append((df[df['Roll Number']==idi]['Phone Number'].values[0]))
            names.append((df[df['Roll Number']==idi]['Names'].values[0]))
            
    url = "https://www.fast2sms.com/dev/bulk"
    headers = {'authorization': "9vUsQhlqu5DtGKMYyB4P6WJNdACoSFiaR3jLHbwczmf2VO8Ip7nzYXQxLFt4gIcdmWy29STeOl5EPjbB",
                'Content-Type': "application/x-www-form-urlencoded",
                'Cache-Control': "no-cache",
                }
    
    #print("before sending messages")
    print("Names")
    print(names)
    print("phone numbers")
    print(phone_numbers)
    for num in phone_numbers:
        if num not in unique_phone_:
            unique_phone_.append(num)
    for numbers in unique_phone_:
        #payload = "sender_id=FSTSMS&message= Your Attendance is marked &language=english&route=p&numbers="+str(numbers)
        response = requests.request("POST", url, data=payload, headers=headers)
        #print(response.text) 
        
print("video capture")
# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)
print("loading images")
# Load a sample picture and learn how to recognize it.

roll_01 = face_recognition.load_image_file("1.jpg")
roll_01_face_encoding = face_recognition.face_encodings(roll_01)[0]

# Load a second sample picture and learn how to recognize it.
roll_02 = face_recognition.load_image_file("2.jpg")
roll_02_face_encoding = face_recognition.face_encodings(roll_02)[0]

roll_03 = face_recognition.load_image_file("3.jpg")
roll_03_face_encoding = face_recognition.face_encodings(roll_03)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    roll_01_face_encoding,
    roll_02_face_encoding,
    roll_03_face_encoding
]
known_face_names = [
    "1",
    "2",
    "3"
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
print("starting web cam")
while True:

    # Getting single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to save time 
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image to RGB
    rgb_small_frame = small_frame[:, :, ::-1]

    # process every other frame not the same one 
    if process_this_frame:


        # find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_roll_no = []
        for face_encoding in face_encodings:

            # comparing faces
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            rollno = "Unknown"


            # getting face distance
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                rollno = known_face_names[best_match_index]
            if rollno not in face_roll_no:
                face_roll_no.append(rollno)
                

    process_this_frame = not process_this_frame


    # Displaying results
    for (top, right, bottom, left), rollno in zip(face_locations, face_roll_no):
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box 
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label 
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, rollno, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
print("marking attendance")

uniq_student_ids = []
for ids in face_roll_no:
    uniq_student_ids.append(int(ids))
print("Roll Numbers")
print(uniq_student_ids )
mark_attendance(uniq_student_ids)
# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

video capture
loading images
starting web cam
marking attendance
Roll Numbers
[]
Names
[]
phone numbers
[]


# payload code  line has been commented to save messages.

In [ ]:
!pip install face_recognition